In [42]:
import os
import json
from pathlib import Path

In [43]:
object_detection_dir = 'D:\測試json\物件偵測'
classification_dir = 'D:\測試json\分類器'
destination_dir = 'D:\測試json\合併'
ODList = os.listdir(object_detection_dir)
CList = os.listdir(classification_dir)

In [44]:
strCList = []
strList = []
all_dict = {}
s = {}

In [45]:
for file in ODList:            
    if file.endswith(".json"):
        with open(os.path.join(object_detection_dir,file), 'r') as reader:
            ODjf = json.loads(reader.read())
        jf = str(ODjf).strip('}').strip('{')
        newjf = str(jf).split(":")
        # org_name為檔名
        org_name = file.split(".")[0]
        for i in range(0,len(newjf)):
            if ',' in str(newjf[i]) and '[' not in str(newjf[i]):
                cut = str(newjf[i]).split(",")
                strList.append(cut[0].lstrip())
                strList.append(cut[1].lstrip())
            elif '}' in str(newjf[i]):
                cut = str(newjf[i]).split("}")
                strList.append(cut[0].lstrip())
                cut_1 = cut[1].split(",")
                strList.append(cut_1[1].lstrip())
            elif '[' in str(newjf[i]) and '}' not in str(newjf[i]):    
                strList.append(newjf[i].lstrip())
            elif '{' in str(newjf[i]):
                cut = str(newjf[i]).split("{")
                strList.append(cut[1].lstrip())
            else:
                strList.append(newjf[i].lstrip())                                  
            
        for file in CList:    
            # name[0]判斷是哪一張圖切下來的 、 item[0]判斷是哪一個 item
            name = file.split("_")   
            if file.endswith(".json") and name[0] == org_name:
                with open(os.path.join(classification_dir,file), 'r') as reader:
                    Cjf = json.loads(reader.read())
                jf = str(Cjf).strip('}').strip('{')
                newjf = str(jf).split(":")  
                item = name[1].split(".")
                strCList.append(item[0].lstrip())
                for i in range(0,len(newjf)):
                    if ',' in str(newjf[i]) :
                        cut = str(newjf[i]).split(",")                  
                        strCList.append(cut[0].lstrip())
                        strCList.append(cut[1].lstrip())
                    else:
                        strCList.append(newjf[i].lstrip())

        t = int( (len(strList) - 2)/7 )
        
        all_dict[strList[0]] = strList[1]  
        
        for i in range(0,t): 
            s[strList[3+7*i]] = strList[4+7*i]
            s[strList[5+7*i]] = strList[6+7*i]
            s[strList[7+7*i]] = strList[8+7*i]
            # 判斷此分類器 json是哪一個 item的
            for n in range(0,t):
                if strCList[n*5] == s["'Class'"].replace("'",'').lstrip() :
                    s[strCList[5*n+1]] = strCList[5*n+2]
                    s[strCList[5*n+3]] = strCList[5*n+4]  
            
            all_dict[strList[2+7*i]] = s
            s = {}
        
        str_all_dict = str(all_dict).replace("'",'')
        all_dict = eval(str_all_dict)
        filename = 'final_' + org_name + '.json'
    
        with open(os.path.join(destination_dir,filename), "w") as f:
            json.dump(all_dict, f) 
            
        strCList = []
        strList = []
        all_dict = {}